# Logbook and all things for Monte-Carlo Minimizor-Maximization


# Algorithm

1. Monte-Carlo Descent method to search for optimal point

-   Use a gradient-based local optimization method
    -   This requires auto-grad by Jax/PyTorch/TensorFlow.
    -   Use Jax for now.
-   Use MCTD to balance exploration and exploitation of the search space

2. Learn the lower bound of the objective function

-   Use a quadratic model to approximate the lower bound of the objective function
-   The learning is as a linear regression problem: finding `A`, such that `Ax <= y`

3. Use interval computation to estimate if the optimal point is found within a small box

-   Use ibex to compute the interval of the objective function

4. Use branch-and-bound to split the search space into smaller boxes

-   Better approx -> better pruning
-   Prunning ambig box -> high priority


## Pseudo code

```
MCMM: (function, box, local_opt, budget)
    create a root node
    while in budget:
        random samples to create new nodes (new step)
        select a path from root to leaf L by UCT
        do local opt on L
```

```
initialize a box B




initialize count to 0
for each element e in the array:
    if e is greater than 10:
        increment count by 1
return count
```


## Problems to solve

Use an toy example  
$\min( 1000*(x+1)^2-3, (x-1)^2-1)$

-   During MCTD optimization, we have
    -   A list of samples from each trajectory
    -   A best-found point (x=1) from 1 or more nodes (but it is not the global min)
    -   0 or more local best-found from 1 or more nodes

3. Use interval computation to estimate if the optimal point is found within a small box

-   interval + function -> Ibex -> interval (can't tell directly when the formula is sat)
    -   Use IBEX to find a small box B' that $f == lb_f$ may occur in
-   Better LB, better pruning?
    -   IBEX works better with quadratic instead of linear.
    -   The best lb to learn is a function that is NEITHER tightly lb to original function NOR completely flat
-   Learned small box B', restart by another MCTD?
    -   No. Sample within the box for another node on MCTD
    -   We need a global LB for the original function, not a piecewise LB.


# Benchmark Settings


## Datasets

1. Synthetic functions

-   [SFU](https://www.sfu.ca/~ssurjano/optimization.html)
-   [Kyoto](http://www-optima.amp.i.kyoto-u.ac.jp/member/student/hedar/Hedar_files/TestGO_files/Page364.htm)

2. Coconut Project (Maybe too old?)

-   [Coconut](https://arnold-neumaier.at/glopt/coconut/Benchmark/Benchmark.html)

3. Real Large Scale Global Optimization

-   [CEC](https://github.com/P-N-Suganthan/2022-SO-BO/blob/main/CEC2022%20TR.pdf)

4. Engineering problems

-   [Matlab](https://www.mathworks.com/matlabcentral/fileexchange/124810-benchmark-problems?s_tid=FX_rc1_behav)

5. Optimization of Black-Box Functions

-   [Black-Box](https://github.com/christiangeissler/gradoptbenchmark/tree/master/)

6. Sets of test problems for MINLP

-   [MINLP](https://www.minlp.com/nlp-and-minlp-test-problems)


## Baselines

1. [Gurobi]() - installed
2. [Scipy.optimize](https://docs.scipy.org/doc/scipy/reference/optimize.html) - installed
3. [PyGMO](https://esa.github.io/pagmo2/docs/cpp/algorithms/nlopt.html) - TODO
4. [BARON](https://sahinidis.coe.gatech.edu/baron) - can do via [NEOS](https://neos-server.org/neos/)

-   BARON guarantees to provide global optima under fairly general assumptions, but cannot handle constraints containing a goniometric function, an if-then-else statement or a reference to an external function. BARON requires that all nonlinear variables and expressions in the mathematical program are bounded from below and above. [source](https://documentation.aimms.com/platform/solvers/baron.html)

4. Gradient based ? check if there are in `scipy.optimize` or `pygmo`
5. Simulated Annealing ? check if there are in `scipy.optimize` or `pygmo`

6. [Matlab FMinCon](https://www.mathworks.com/help/optim/ug/fmincon.html) - installed, but least priority because it is not free and not in python


# Results Memo


Example of a result table

| Function Name      | Best Result | Best Result Time | Average Result       | Average Result Time |
| ------------------ | ----------- | ---------------- | -------------------- | ------------------- |
| Levy - 10d         | 1.68e-12    | ~ 20s            | 1.44 +- 2.14         | ~ 20s               |
| Levy - 50d         | 4.86e-10    | ~ 60s            | 7.07 +- 11.08        | ~ 60s               |
| Levy - 100d        | 3.64e-08    | ~ 120s           | 20.31 +- 30.28       | ~ 120s              |
| Levy - 500d        | 1.34e-04    | ~ 1200s          | 117.43 +- 178.05     | ~ 1200s             |
| Ackley - 10d       | 1.73e-09    | ~ 20s            | 7.33 +- 12.45        | ~ 20s               |
| Ackley - 50d       | 4.86e-08    | ~ 60s            | 64.99 +- 105.35      | ~ 60s               |
| Ackley - 100d      | 2.74e-06    | ~ 120s           | 489.23 +- 678.59     | ~ 120s              |
| Ackley - 500d      | 3.45e-01    | ~ 1200s          | 56929.07 +- 87258.25 | ~ 1200s             |
| Schwefel - 10d     | 2.81e-10    | ~ 20s            | 106.39 +- 147.55     | ~ 20s               |
| Schwefel - 50d     | 1.10e-08    | ~ 60s            | 4433.81 +- 6447.66   | ~ 60s               |
| Schwefel - 100d    | 1.22e-06    | ~ 120s           | 23868.55 +- 32918.32 | ~ 120s              |
| Schwefel - 500d    | 4.81e+02    | ~ 1200s          | 1.29e+08 +- 1.93e+08 | ~ 1200s             |
| Drop Wave - 10d    | 1.11e-05    | ~ 20s            | 2.48 +- 3.69         | ~ 20s               |
| Drop Wave - 50d    | 1.25e-04    | ~ 60s            | 36.59 +- 55.83       | ~ 60s               |
| Drop Wave - 100d   | 1.11e-03    | ~ 120s           | 139.46 +- 201.42     | ~ 120s              |
| Drop Wave - 500d   | 2.54e+01    | ~ 1200s          | 37845.81 +- 60035.78 | ~ 1200s             |
| Michalewicz - 10d  | 1.03e-02    | ~ 20s            | 2.17 +- 3.22         | ~ 20s               |
| Michalewicz - 50d  | 6.21e+00    | ~ 20s            |
| Michalewicz - 100d |
| Michalewicz - 500d |


## Example of results from different random seeds

-   For Levy 50d

```
best_found_history = [
11.162158012390137,
4.864455505071419e-10,
34.244049072265625,
2.0835724812151568e-10,
8.990206718444824,
0.08952824026346207,
0.2685847580432892,
1.8039302825927734,
]
```
